In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI()  # Remove parentheses here

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the anoswer just say you don't know don't make it up:\n\n{context}"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

chain.invoke("Describe Victory Mansions.")

AIMessage(content='Victory Mansions is a building with glass doors that Winston Smith enters on a cold day in April. The hallway smells of boiled cabbage and old rag mats. There is a coloured poster of a man\'s face tacked to the wall. The building has a faulty lift, and Winston climbs seven flights of stairs to his flat. The poster with the enormous face is visible on each landing, with the caption "BIG BROTHER IS WATCHING YOU" underneath it. Inside the flat, there is a telescreen on the wall that cannot be completely shut off.')